## 3D Point Cloud Simulation of 2D KITTI LiDAR Dataset using Open3D

### Code written by Pranav Durai

In [1]:
# Import library functions
import os
import time
import requests
import math

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

In [2]:
# Install necessary packages - Matplotlib, Numpy, and Open3D
!pip install matplotlib numpy open3d   

In [3]:
# Function definition to download the dataset
def download_dataset(url, local_filename):

    # Update Dropbox link to force download
    if "www.dropbox.com" in url and "?dl=0" in url:
        url = url.replace("?dl=0", "?dl=1")
    
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded and saved as {local_filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

In [4]:
# Download 2D KITTI Depth Frames Dataset 
download_dataset('https://www.dropbox.com/scl/fi/wfg0ta7kx57be15hw40wl/archive.zip?rlkey=fei6eqeucmbcbkw478dfsy7qg&dl=1', 'archive.zip')

### Read 2D Depth Image

The process begins by loading an image using `plt.imread` from the `matplotlib` library, with the image data being stored in the variable `depth_image`. Following this, a critical step involves setting a `depth_image_scaling_factor`, which is essential for converting the normalized depth values in the image into actual distance measurements. These measurements represent the real-world distances from the 3D LiDAR sensor to the objects in the scene. The value of this scaling factor is specifically tailored to align with the characteristics of the KITTI dataset. Lastly, an adjustment is made to the normalization process, transforming the normalized values into a more meaningful representation of actual distances, thereby enhancing the depth data's utility for further processing and analysis.

In [5]:
# Read the 2D Depth Image
def load_depth_image(file_path):
    # Load the depth image
    depth_image = plt.imread(file_path)

    depth_image_scaling_factor = 250.0
    # Assuming the depth image is normalized, we may need to scale it to the actual distance values
    # This scaling factor is dataset-specific; you'll need to adjust it based on the KITTI dataset documentation
    depth_image *= depth_image_scaling_factor
    
    return depth_image

### Load and Process Frames
This function initializes a `point_clouds` list and iterates over depth image files in a specified `directory`, processing them in alphabetical or numerical order using `sorted(os.listdir(directory))`. It checks each file to ensure it is a PNG image with `filename.endswith('.png')` for consistent processing of depth image files. For each PNG file, the function constructs its full path, loads the depth image using `load_depth_image`, and then converts it into a point cloud using `depth_image_to_point_cloud`. This conversion transforms the 2D depth information into a 3D point cloud representation, which is appended to the `point_clouds` list. Finally, the function returns this list of point clouds after processing all PNG images in the directory.

In [6]:
def load_and_process_frames(directory):
    point_clouds = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.png'):  # Check for PNG images
            file_path = os.path.join(directory, filename)
            depth_image = load_depth_image(file_path)
            point_cloud = depth_image_to_point_cloud(depth_image)
            point_clouds.append(point_cloud)
    return point_clouds

### Converting 2D Depth Frames into 3D Point Cloud
This function processes a 2D depth image (`depth_image`) by first calculating horizontal (`h_angles`) and vertical (`v_angles`) angles using `np.linspace` within the specified horizontal (`h_fov`) and vertical (`v_fov`) fields of view, and converting these angles from degrees to radians. It then reshapes the horizontal angles to match the depth image's column count and the vertical angles to match its row count, allowing for broadcasting in subsequent calculations. Using trigonometric operations, it computes the x, y, and z coordinates for each point in the point cloud from the depth values, applying the angles. Points outside a specified distance range (`d_range`) are filtered out to maintain data relevance. Finally, the valid coordinates are stacked to form a 3D point cloud, suitable for three-dimensional visualization and analysis.

In [7]:
def depth_image_to_point_cloud(depth_image, h_fov=(-90, 90), v_fov=(-24.9, 2.0), d_range=(0,100)):
    # Adjusting angles for broadcasting
    h_angles = np.deg2rad(np.linspace(h_fov[0], h_fov[1], depth_image.shape[1]))
    v_angles = np.deg2rad(np.linspace(v_fov[0], v_fov[1], depth_image.shape[0]))

    # Reshaping angles for broadcasting
    h_angles = h_angles[np.newaxis, :]  # Shape becomes (1, 1440)
    v_angles = v_angles[:, np.newaxis]  # Shape becomes (64, 1)

    # Calculate x, y, and z
    x = depth_image * np.sin(h_angles) * np.cos(v_angles)
    y = depth_image * np.cos(h_angles) * np.cos(v_angles)
    z = depth_image * np.sin(v_angles)

    # Filter out points beyond the distance range
    valid_indices = (depth_image >= d_range[0]) & (depth_image <= d_range[1])
    
    # Apply the mask to each coordinate array
    x = x[valid_indices]
    y = y[valid_indices]
    z = z[valid_indices]

    # Stack to get the point cloud
    point_cloud = np.stack((x, y, z), axis=-1)

    return point_cloud

### Simulate Point Cloud Representation
The `animate_point_clouds` function initializes an Open3D Visualizer (`vis`) with a black background to display a list of point cloud data, `point_clouds`, where each list element represents a frame in the animation. It creates a new Open3D PointCloud object (`point_cloud`) and sets its points to the first frame for initial display. During the simulation loop, it cycles through the frames, updating the point cloud's points and the visualizer's geometry based on a set `update_interval`. User interactions and rendering updates are managed with `vis.poll_events()` and `vis.update_renderer()`. The simulation terminates and closes the visualizer window when `vis.poll_events()` returns False, indicating the user has closed the window.

<img src="https://www.dropbox.com/scl/fi/uz0t97vn85u0ej9u7dfbh/POV-view-1.gif?rlkey=resp0390b0feqou04tiz8l0e6&dl=1" width="800">

In [8]:
def animate_point_clouds(point_clouds):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Set background color to black
    vis.get_render_option().background_color = np.array([0, 0, 0])

    # Initialize point cloud geometry
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(point_clouds[0])
    vis.add_geometry(point_cloud)

    frame_index = 0
    last_update_time = time.time()
    update_interval = 0.25  # Time in seconds between frame updates

    while True:
        current_time = time.time()
        if current_time - last_update_time > update_interval:
            # Update point cloud with new data
            point_cloud.points = o3d.utility.Vector3dVector(point_clouds[frame_index])
            vis.update_geometry(point_cloud)

            # Move to the next frame
            frame_index = (frame_index + 1) % len(point_clouds)
            last_update_time = current_time

        vis.poll_events()
        vis.update_renderer()

        if not vis.poll_events():
            break

    vis.destroy_window()

In [ ]:
# Directory containing the depth image files
directory = 'archive/2011_09_30_drive_0028_sync/2011_09_30_drive_0028_sync/2011_09_30/2011_09_30_drive_0028_sync/velodyne_points/depth_images'

# Load and process the frames
point_clouds = load_and_process_frames(directory)

# Simulate the point clouds
animate_point_clouds(point_clouds)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
